In [ ]:
# pip install fastapi uvicorn sqlalchemy mysql-connector-python
# pip install geopy

In [13]:
import mysql.connector
from mysql.connector import Error
from math import radians, sin, cos, sqrt, atan2

In [4]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sqlalchemy import create_engine, Table, MetaData, select
from sqlalchemy.sql import text
from typing import List, Dict
from geopy.distance import great_circle

app = FastAPI()

# Database configuration
DATABASE_URL = "mysql+mysqlconnector://root:@dmintce@localhost/workshop"
engine = create_engine(DATABASE_URL, echo=True)
metadata = MetaData()

In [10]:
# Helper function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2) -> float:
    return round(great_circle((lat1, lon1), (lat2, lon2)).km, 2)
 

In [71]:
mysql_host = 'localhost'
mysql_db = 'workshop'
mysql_user = 'root'
mysql_password = '@dmintce'


In [121]:
def get_warehouses_within_distance(lat, lon, max_distance_km):
    try:
        connection = mysql.connector.connect(
            host= mysql_host,
            database= mysql_db,
            user= mysql_user,
            password=  mysql_password
        )
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            
            # Query to get all warehouses
            query = """
            SELECT site, latitude, longitude 
            FROM warehouse
            """
            cursor.execute(query)
            warehouses = cursor.fetchall()
            
            # Filter warehouses based on distance
            nearby_warehouses = []
            for warehouse in warehouses:
                dist = calculate_distance(lat, lon, warehouse['latitude'], warehouse['longitude'])
                if dist <= max_distance_km:
                    nearby_warehouses.append(warehouse['site'])
            
            return nearby_warehouses

    except Error as e:
        print(f"Error: {e}")
        return None

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()



In [178]:
def get_shipment(lat, lon, max_distance_km,car_license):
    try:
        connection = mysql.connector.connect(
            host= mysql_host,
            database= mysql_db,
            user= mysql_user,
            password=  mysql_password
        )
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            
            # Query to get all warehouses
            query = """
            SELECT site, latitude, longitude 
            FROM warehouse
            """
            cursor.execute(query)
            warehouses = cursor.fetchall()
            
            # Filter warehouses based on distance
            warehouses_site = []
            for warehouse in warehouses:
                dist = calculate_distance(lat, lon, warehouse['latitude'], warehouse['longitude'])
                if dist <= max_distance_km:
                    warehouses_site.append(warehouse['site'])
            result = ' '
            result = ', '.join(f"'{item}'" for item in  warehouses_site  )

            # Create the SQL query text
            shipment_query = ''
            
            if result != ' ':
                shipment_query = f"""SELECT s.ship_point, p.province 
                FROM shipment  s 
                inner join province p
                on s.postcode = p.postcode
                WHERE car_license = '{car_license}'
                  and load_stat = '' and ship_point in ({result})"""
            # print(shipment_query)
            cursor.execute(shipment_query)
            shipment = cursor.fetchall()
            return shipment

    except Error as e:
        print(f"Error: {e}")
        return None

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

In [ ]:
# Example usage
#Vitory monament
input_latitude  = 13.45536
input_longitude  = 100.53830
# #Bang naum paw
input_latitude  = 13.69273
input_longitude = 100.89219
distance_km = 20

shipment = get_shipment(input_latitude, input_longitude, distance_km,'กท1')
shipment